In [16]:
# REST API

import requests
import pandas as pd # 1분봉 조회하기
import numpy as np
import pyupbit
from sklearn.preprocessing import StandardScaler  


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
def window_splitted_data_generater(df, window_size):
    
    x_list = []
    y_list = []
    
    for i in range(len(df) - window_size):
        x_list.append(df.iloc[i:i+window_size, : -1])
        y_list.append(df.iloc[i + window_size, -1])
        
    
    x_list = np.array(x_list)
    y_list = np.array(y_list)
        
    return x_list, y_list

In [8]:
#1. Load data
url = "https://api.upbit.com/v1/market/all"

resp = requests.get(url)
data = resp.json()


df = pyupbit.get_ohlcv("KRW-BTC", "minute1")
df.head()
df.shape

(200, 6)

In [ ]:
# data quality check

In [9]:
#2. data normalization


scaler = StandardScaler()
df.iloc[:, 0:4] = scaler.fit_transform(df.iloc[:, 0:4])  

In [10]:
#3. EDA

In [12]:
# Split train, test data
SPLIT = int(0.7*len(df)) 
train_df = df[:SPLIT]
test_df = df[SPLIT:]

In [13]:
#3. Prepare X, y according to prediction window.

window_size = 10 # predicting close price using data from 10 previous minutes
x_train, y_train = window_splitted_data_generater(train_df, window_size)
x_test, y_test = window_splitted_data_generater(test_df, window_size)

In [14]:
# model definition

import torch.nn as nn

class CoinLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size = 50, num_layers = 1, batch_first= True)
        self.linear = nn.Linear(50,1)
        
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

In [15]:
def train_step(x_batch, y_batch, optimizer, loss_fn, model):
    
    optimizer.zero_grad()
    
    # forward pass
    y_pred = model(x_batch)
    
    # calculate loss of the batch
    loss = loss_fn(y_pred, y_batch)
    
    # backward propagation
    loss.backward()
    
    # update weights
    optimizer.step()
    
    return loss.item() 

In [ ]:
# create custom dataset 


In [ ]:
# create data loader

In [17]:
# assgin model, optimizer, loss_fn, data_loader
model = CoinLSTM()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
train_data_loader =  data.DataLoader(data.TensorDataset(x_train, y_train), shuffle=True, batch_size=8)
val_data_loader = data.DataLoader(data.TensorDataset(x_test, y_test), batch_size = 8)

tr_loss_history = []
val_loss_history = []

n_epochs = 100

for epoch in range(n_epochs):
    model.train()  # Important to set training mode
    losses = []
    
    for X_batch, y_batch in train_data_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        loss = train_step(X_batch, y_batch)
        tr_loss_history.append(loss)
    
    # Add validation step after each epoch
    model.eval()
    with torch.no_grad():
        # Uses loader to fetch one mini-batch for validation
        for x_val, y_val in val_data_loader:
            # Again, sends data to same device as model
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            
            model.eval()

            yhat = model(x_val)

            val_loss = loss_fn(y_val, yhat)
            val_loss_history.append(val_loss.item())

AttributeError: 'list' object has no attribute 'DataLoader'